<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0530/0530_05_%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81_LSA%26LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## LSA(잠재 의미 분석)

사이킷런에서는 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스그룹 데이터를 제공합니다. 여기서는 LSA를 사용해서 문서의 수를 원하는 토픽의 수로 압축한 뒤에 각 토픽당 가장 중요한 단어 5개를 출력하는 실습으로 토픽 모델링을 수행합니다.

- 데이터 : https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

In [5]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

RuntimeError: ignored

ImportError: ignored

In [ ]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [ ]:
print(dataset.target_names) # 분류 카테고리

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
print(dataset.DESCR) # 데이터 설명

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [ ]:
# 첫번째 샘플 확인하기 
print(documents[0])

Well i'm not sure about the story nad it did seem biased. What
I disagree with is your statement that the U.S. Media is out to
ruin Israels reputation. That is rediculous. The U.S. media is
the most pro-israeli media in the world. Having lived in Europe
I realize that incidences such as the one described in the
letter have occured. The U.S. media as a whole seem to try to
ignore them. The U.S. is subsidizing Israels existance and the
Europeans are not (at least not to the same degree). So I think
that might be a reason they report more clearly on the
atrocities.
	What is a shame is that in Austria, daily reports of
the inhuman acts commited by Israeli soldiers and the blessing
received from the Government makes some of the Holocaust guilt
go away. After all, look how the Jews are treating other races
when they got power. It is unfortunate.



In [ ]:
# 카테고리 확인하기
print(dataset.target_names[dataset.target[0]])

talk.politics.mideast


### 1. 텍스트 전처리 

In [ ]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")

# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [ ]:
news_df['clean_doc'][0]

"well i'm not sure about the story nad it did seem biased. what\ni disagree with is your statement that the u.s. media is out to\nruin israels reputation. that is rediculous. the u.s. media is\nthe most pro-israeli media in the world. having lived in europe\ni realize that incidences such as the one described in the\nletter have occured. the u.s. media as a whole seem to try to\nignore them. the u.s. is subsidizing israels existance and the\neuropeans are not (at least not to the same degree). so i think\nthat might be a reason they report more clearly on the\natrocities.\n\twhat is a shame is that in austria, daily reports of\nthe inhuman acts commited by israeli soldiers and the blessing\nreceived from the government makes some of the holocaust guilt\ngo away. after all, look how the jews are treating other races\nwhen they got power. it is unfortunate.\n"

### 2. TF-IDF 벡터화

In [ ]:
# TDM(문장을 행렬화)이 있어야 LSA 실행가능
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (11314, 1000)


### 3. LSA
- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

위에서 만든 TF-IDF 행렬을 SVD로 분해해보겠습니다. 
이때, 토픽의 숫자를 정해야 하는데, 기존 뉴스그룹 데이터의 카테고리가 20개이기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽모델링을 시도해보겠습니다. 
- n_components : 토픽의 숫자

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
# n_ite = 100 : 100번을 돌리겠다
# n_components : 토픽개수
svd_model.fit(X)

TruncatedSVD(n_components=20, n_iter=100, random_state=122)

In [ ]:
svd_model.components_.shape # V.T : 토픽의 수 t x 단어의 수 1000 = 각 토픽에 의미를 부여하는 단어 1000개가 있다
#components_ : 결과값, 앞의 20 : 토픽이 20개 있다,  

(20, 1000)

In [ ]:
svd_model.components_ # V.T : 토픽의 수 t x 단어의 수 1000

array([[ 0.02424454,  0.02500425,  0.00458373, ...,  0.04546442,
         0.01314782,  0.01570363],
       [ 0.04285648, -0.00563424,  0.00628099, ..., -0.02209406,
        -0.00339099, -0.01430937],
       [-0.03819118, -0.04036904, -0.0032203 , ..., -0.00537898,
        -0.02214086, -0.01375735],
       ...,
       [ 0.0478484 ,  0.01712013,  0.0025385 , ..., -0.0206397 ,
         0.00200983,  0.00668018],
       [ 0.03086541,  0.01323   ,  0.00016956, ...,  0.01283668,
        -0.00319745,  0.00720587],
       [-0.01471904, -0.00729477,  0.00027112, ...,  0.01959091,
        -0.01898329, -0.00121688]])

In [ ]:
# 첫번째 토픽의 단어 분포(가중치, 크기비교한 것)
svd_model.components_[0]

array([2.42445435e-02, 2.50042513e-02, 4.58372888e-03, 6.05308797e-03,
       3.06838802e-03, 4.03690728e-03, 1.91830340e-04, 8.46496618e-05,
       4.53683244e-02, 2.62023513e-02, 2.01240980e-02, 2.63458864e-02,
       5.22876347e-03, 1.50616059e-02, 2.11991716e-02, 8.49807217e-04,
       2.85425271e-02, 2.68827255e-02, 1.63969965e-02, 1.33505967e-02,
       1.10674764e-02, 8.91026002e-03, 6.63331123e-03, 1.26684217e-02,
       1.97920396e-02, 7.91802071e-06, 6.82806218e-03, 8.72045465e-05,
       3.32799849e-02, 1.35231902e-02, 1.25681013e-02, 1.01253511e-02,
       9.92512957e-03, 1.88291013e-02, 2.21107415e-02, 8.95266370e-03,
       8.82370442e-03, 7.85006237e-03, 7.84745226e-03, 6.43862326e-03,
       4.13250943e-06, 2.22723239e-06, 2.67285876e-02, 1.29474097e-02,
       6.19236205e-03, 1.30772423e-02, 9.30879856e-03, 5.09795706e-03,
       5.21088333e-05, 8.08140016e-03, 3.20954052e-03, 4.04155295e-03,
       4.50428786e-03, 4.25226076e-03, 9.21294394e-03, 1.35274913e-04,
      

In [ ]:
# 숫자들 중 크기 순서대로 정렬 >> 인덱스
svd_model.components_[0].argsort()[::-1][:10]
#크기 순서대로 정렬해서 인덱스를 알려줌 : argsort()
#argsort()[::-1]: 내림차순 정렬한 것의 인덱스
#[:10] : 슬라이싱

In [ ]:
vectorizer.get_feature_names_out()[450] # 450번째 단어

In [ ]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names_out()

In [ ]:
# 첫번째 토픽의 단어들 중 가장 중요한 단어 
n = 10 
idx_10 = svd_model.components_[0].argsort()[::-1][:n]
terms[idx_10]

array(['just', 'don', 'like', 'know', 'people', 'think', 'does', 'good',
       'use', 'time'], dtype=object)

In [ ]:
terms = vectorizer.get_feature_names_out() # 단어 집합. 1,000개의 단어가 저장됨.>> 20개의 토픽마다 중요한 단어가 다르게 저장됨

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[::-1][:n]])
        # 단어이름 : feature_names
        # 중요도 반올림 : topic[i].round(5)
        # topic.argsort()[::-1][:n] : 토픽별로 내림차순 정렬해서 n개만 보겠다
get_topics(svd_model.components_,terms)
#components : 부품, 요소
#components_ : SVD 중에 차원을(VT)에 의미를 부여했다
#토픽은 단어의 유사도가 아니라 같이 등장하는 횟수가 많아서 같은 차원에 묶였다고 보는 것이 좋음

Topic 1: [('just', 0.18631), ('don', 0.18599), ('like', 0.17994), ('know', 0.17157), ('people', 0.16884)]
Topic 2: [('thanks', 0.25292), ('windows', 0.2476), ('card', 0.14837), ('drive', 0.13738), ('use', 0.13404)]
Topic 3: [('god', 0.57224), ('jesus', 0.19354), ('does', 0.19288), ('thanks', 0.15636), ('windows', 0.12981)]
Topic 4: [('key', 0.26592), ('use', 0.21321), ('people', 0.18792), ('government', 0.18546), ('chip', 0.14362)]
Topic 5: [('know', 0.36888), ('don', 0.28779), ('just', 0.24189), ('does', 0.21113), ('thanks', 0.19486)]
Topic 6: [('edu', 0.41317), ('thanks', 0.24944), ('mail', 0.17055), ('know', 0.15304), ('com', 0.10533)]
Topic 7: [('drive', 0.36621), ('thanks', 0.15455), ('car', 0.15276), ('scsi', 0.14102), ('know', 0.13672)]
Topic 8: [('edu', 0.55147), ('just', 0.32737), ('don', 0.15035), ('soon', 0.12199), ('think', 0.11496)]
Topic 9: [('key', 0.36329), ('god', 0.2402), ('just', 0.2294), ('chip', 0.19286), ('use', 0.16339)]
Topic 10: [('car', 0.41773), ('like', 0.21

## LDA
- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

### 1. 벡터화

전처리는 LSA과정과 동일함으로 생략하고 바로 벡터화부터 진행하겠습니다. 

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TF-IDF 행렬의 크기 : (11314, 1000)


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=20, random_state=777)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_top = lda_model.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(lda_model.components_)
print(lda_model.components_.shape) 

[[ 0.05        0.05000001  0.05000003 ...  3.98732417  0.44184971
   0.05      ]
 [ 0.06182269  0.12816121  0.05       ...  4.89279057  0.11968496
   0.34172264]
 [ 0.05        0.05        0.05       ...  7.05387199  0.05
   0.05      ]
 ...
 [46.15071523  7.01628252  1.72141864 ...  0.05000006  0.05000042
   0.05      ]
 [ 0.05        0.05        0.05       ...  0.05        0.05
   0.05      ]
 [ 0.87249043  4.60691604  0.05       ...  7.08265123  0.4425896
   0.05      ]]
(20, 1000)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('file', 61.51), ('window', 61.3), ('problem', 52.0), ('use', 46.67), ('windows', 46.17)]
Topic 2: [('key', 67.67), ('chip', 48.13), ('encryption', 40.33), ('clipper', 39.28), ('keys', 35.68)]
Topic 3: [('edu', 31.4), ('deleted', 22.77), ('soon', 20.38), ('stuff', 16.41), ('mike', 13.31)]
Topic 4: [('just', 18.07), ('water', 14.87), ('800', 11.72), ('jewish', 10.77), ('book', 9.93)]
Topic 5: [('windows', 74.59), ('thanks', 72.21), ('card', 48.68), ('does', 45.31), ('know', 44.22)]
Topic 6: [('bike', 57.75), ('dod', 22.27), ('right', 15.92), ('company', 13.56), ('road', 12.37)]
Topic 7: [('game', 84.73), ('team', 75.54), ('year', 67.24), ('games', 54.97), ('players', 47.68)]
Topic 8: [('god', 70.24), ('people', 63.03), ('israel', 55.16), ('jesus', 48.86), ('jews', 43.6)]
Topic 9: [('drive', 45.35), ('scsi', 45.17), ('ide', 25.17), ('controller', 24.56), ('drives', 22.47)]
Topic 10: [('hp', 16.3), ('good', 10.26), ('know', 9.4), ('let', 8.37), ('things', 7.85)]
Topic 11: [('mac

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 시각화
# LDAvis 는 토픽 모델링에 자주 이용되는 Latent Dirichlet Allocation (LDA) 모델의 학습 결과를 시각적으로 표현하는 라이브러리입니다
!pip install -U pyLDAvis

In [ ]:
# LDA 시각화를 진행해봅시다. 
import pyLDAvis
import pyLDAvis.lda_model 

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda_model = lda_model, 
                       dtm = X, 
                       vectorizer = vectorizer, 
                       mds='tsne')
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LDAvis 는 두 가지 정보를 출력합니다. 첫째는 HTML 의 좌측에 출력되는 topic 의 2 차원 embedding vector 입니다. 비슷한 위치에 존재하는 토픽들은 서로 비슷한 문맥을 지니고 있습니다. 그리고 둘째는 오른쪽에 출력되는 각 토픽의 키워드 입니다.

좌측의 원들은 각각의 20개의 토픽을 나타냅니다.
각 원과의 거리는 각 토픽들이 서로 얼마나 다른지를 보여줍니다.만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미입니다.
각 원을 클릭하면 우측에 해당 토픽에 관한 정보가 나타납니다

LDAvis 에서는 두 관점의 중요도를 사람이 직접 정할 수 있도록 하였습니다. 우측상단에 λ 라는 값이 있습니다. 이 값은 [0, 1] 사이에서 조절가능한 값입니다. 그리고 그 아래 키워드 랭킹은 다음의 점수로 계산됩니다. λ를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미이며, λ 를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미입니다. https://lovit.github.io/nlp/2018/09/27/pyldavis_lda/

---
- Reference : https://wikidocs.net/24949